In [1]:
import pickle

with open("data/product.pickle", 'rb') as p:
    product = pickle.load(p)
with open("data/purchase.pickle", 'rb') as p:
    purchase = pickle.load(p)
with open("data/user.pickle", 'rb') as u:
    user = pickle.load(u)

In [2]:
# 구매 데이터 중 상품 정보가 있는 것만 저장
df = purchase.merge(product, how = 'inner', right_on='상품명', left_on='상품')
df.shape

(5427, 31)

In [3]:
#리뷰자별 리뷰 전문
review = df.groupby('리뷰자')['본문'].sum().reset_index()
print(review.shape)

(3576, 2)


In [ ]:
#특수문자 삭제
review['본문'] = review['본문'].replace(r'[^0-9가-힣a-zA-Z\s]', '', regex=True)
review['본문'] = review['본문'].replace('\n', '', regex=True)
review['본문'] = review['본문'].replace('\xa0', ' ', regex=True)
review = review[review['본문'] != ''].reset_index(drop=True)

In [5]:
length = list()

for i in review['본문']:
    length.append(len(i))

# 리뷰 길이로 인한 오류 발생 확인 : tokenizer.encode 사용 시
print("max length of review : ", max(length))
print("min length of review : ", min(length))

max length of review :  2713
min length of review :  14


In [8]:
#KoBART 이용
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [20]:
def generate_summary(text):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    inputs = tokenizer.batch_encode_plus([text], return_tensors="pt", max_length=1024, truncation=True).to(device)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    summary_ids = model.to(device).generate(input_ids, attention_mask=attention_mask, max_length=512, num_beams=2)
    summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
    return summary

In [21]:
import numpy as np
review['요약문'] = np.nan

for i,j in enumerate(review['본문']):
    try:
        summerization = generate_summary(j)
        review.loc[i, '요약문'] = summerization
        if i % 10 == 0:
            print(i, "번째 사용자 요약 완료")
    except IndexError:
        print(i, "th review is tooo long!")

0 번째 사용자 요약 완료
10 번째 사용자 요약 완료
20 번째 사용자 요약 완료
30 번째 사용자 요약 완료
40 번째 사용자 요약 완료
50 번째 사용자 요약 완료
60 번째 사용자 요약 완료
70 번째 사용자 요약 완료
80 번째 사용자 요약 완료
90 번째 사용자 요약 완료
100 번째 사용자 요약 완료
110 번째 사용자 요약 완료
120 번째 사용자 요약 완료
130 번째 사용자 요약 완료
140 번째 사용자 요약 완료
150 번째 사용자 요약 완료
160 번째 사용자 요약 완료
170 번째 사용자 요약 완료
180 번째 사용자 요약 완료
190 번째 사용자 요약 완료
200 번째 사용자 요약 완료
210 번째 사용자 요약 완료
220 번째 사용자 요약 완료
230 번째 사용자 요약 완료
240 번째 사용자 요약 완료
250 번째 사용자 요약 완료
260 번째 사용자 요약 완료
270 번째 사용자 요약 완료
280 번째 사용자 요약 완료
290 번째 사용자 요약 완료
300 번째 사용자 요약 완료
310 번째 사용자 요약 완료
320 번째 사용자 요약 완료
330 번째 사용자 요약 완료
340 번째 사용자 요약 완료
350 번째 사용자 요약 완료
360 번째 사용자 요약 완료
370 번째 사용자 요약 완료
380 번째 사용자 요약 완료
390 번째 사용자 요약 완료
400 번째 사용자 요약 완료
410 번째 사용자 요약 완료
420 번째 사용자 요약 완료
430 번째 사용자 요약 완료
440 번째 사용자 요약 완료
450 번째 사용자 요약 완료
460 번째 사용자 요약 완료
470 번째 사용자 요약 완료
480 번째 사용자 요약 완료
490 번째 사용자 요약 완료
500 번째 사용자 요약 완료
510 번째 사용자 요약 완료
520 번째 사용자 요약 완료
530 번째 사용자 요약 완료
540 번째 사용자 요약 완료
550 번째 사용자 요약 완료
560 번째 사용자 요약 완료
570 번째 사용자 요약 완료
580 번째 사용자 요약 완료
590 번째 사

In [22]:
review.isna().sum() #요약 안 된 개수 확인

리뷰자    0
본문     0
요약문    0
dtype: int64

In [23]:
with open("data/kobart_summerize.pickle", 'wb') as k:
    pickle.dump(review, k)